<a href="https://colab.research.google.com/github/maddswall/r-cuboulder-sentiment-analysis/blob/main/Bertopic_2016_CU_Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Modern AI Project <br>
NLP analysis on reddit comments from the r/cuboulder community.




In [ ]:
#!pip install nltk
#!pip install spacy
#!pip install sklearn

In [2]:
import pandas as pd

In [5]:
!git clone https://github.com/maddswall/r-cuboulder-sentiment-analysis.git

fatal: destination path 'r-cuboulder-sentiment-analysis' already exists and is not an empty directory.


In [8]:
df = pd.read_csv('/content/r-cuboulder-sentiment-analysis/cu_comments_2016.csv', index_col=0)

In [ ]:
#simple analytics
shape = df.shape

print(f'There are {shape[0]} rows and {shape[1]} columns')

There are 2829 rows and 4 columns


In [ ]:
#take a look at the first few rows
df.head()

,ID,title,date,comment
0,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"as one of the security guards there, the paren..."
1,da859t,Bruh we’re being watched,2019-09-27 23:14:31,A bunch of elementary school kids (or maybe su...
2,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"""And here's the gym where your kid can pretend..."
3,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"As a high school student who toured here, I ca..."
4,da859t,Bruh we’re being watched,2019-09-27 23:14:31,Former Norlin guard here. Sounds like a case o...


In [ ]:
# A look at the posts with the highest comments.
top_post = df.groupby('title').size().reset_index(name='count').sort_values('count', ascending=False)
top_post.head(10)

,title,count
128,Stand Up for CU,72
67,I Fucking Hate Skateboarders On This Campus,65
24,"CU president Mark Kennedy to make $850,000 in ...",50
119,Resurrection Church at Norlin Library,50
154,We have a winner /s,47
86,It do be like that,46
15,Best places to cry on campus?,44
173,YKIYK,37
179,apolgy for bad english,36
177,academic advisors aren't even helpful,35


In [ ]:
#longest comment
df['comment_count'] = df.comment.str.len()
df.sort_values('comment_count', ascending =False).head(10)

,ID,title,date,comment,comment_count
176,dpb355,What to do with a billion dollar budget,2019-10-30 19:00:20,Alright - I did undergrad here and then came b...,3405
1042,bc42fq,Stand Up for CU,2019-04-11 19:30:33,I'd like to share some of my experience as a f...,2008
1077,bc42fq,Stand Up for CU,2019-04-11 19:30:33,I think you’re missing the point of the poster...,1858
1978,al9x2z,What a night.,2019-01-30 06:27:52,Incredible that we had such a similar experien...,1292
2481,cn14xf,New Students: Welcome to CU!,2019-08-07 04:26:54,This is a great thread. A couple of things I ...,1220
1111,bc42fq,Stand Up for CU,2019-04-11 19:30:33,"I actually do oppose genocide, and would regar...",1215
750,6x21fb,I Fucking Hate Skateboarders On This Campus,2017-08-30 20:37:59,**Demagogue**\n\nA demagogue (from Greek δημα...,1188
1102,bc42fq,Stand Up for CU,2019-04-11 19:30:33,>\tWhat are you even talking about?\n\nI’m tal...,1159
1619,df25dj,We have a winner /s,2019-10-08 16:11:33,I really really wish I was there. \n\nI relish...,1137
2484,cn14xf,New Students: Welcome to CU!,2019-08-07 04:26:54,For CU jobs you may visit [their website](http...,982


In [ ]:
#shortest values
short_comments = df.loc[df.comment_count < 5]
print(f'There are {len(short_comments)} comments less than 5 characters in length')
print('\nComment_count short frequency')
short_comments.comment_count.value_counts()

There are 162 comments less than 5 characters in length

Comment_count short frequency


1    95
4    38
3    24
2     5
Name: comment_count, dtype: int64

In [ ]:
short_comments.head(5)

,ID,title,date,comment,comment_count
24,dflspt,If you switch between them enough times the in...,2019-10-09 19:11:31,OMFG,4
96,cxbed5,YKIYK,2019-08-30 03:24:18,No,2
138,ehrz5u,apolgy for bad english,2019-12-30 20:54:45,F,1
140,ehrz5u,apolgy for bad english,2019-12-30 20:54:45,F,1
141,ehrz5u,apolgy for bad english,2019-12-30 20:54:45,F,1


In [ ]:
#removing posts that were removed or deleted by the user of the moderator
df = df[~((df['comment']=='[removed]') | (df['comment']=='[deleted]'))]

In [ ]:
df.shape

(2679, 5)

##Cleaning the Data and Removing Stop Words

In [ ]:
#removing punctuation from the comments column
import string
import re

#drop rows that contain [removed] or [deleted]
df.drop(df.loc[(df.comment == '[removed]') | (df.comment == '[deleted]')]\
        .index, inplace=True)

#drop comments less than 5 characters in length
df = df[df['comment_count'] >= 5]

#define a string of all punctuation characters
punct = string.punctuation
#define a regex to look for hyperlinks
url_regex = re.compile(r'http\S+|www.\S+')
#list of profanity to remove
cuss_words = ['fuck', 'shit', 'asshole', 'bitch', 'cunt', 'dick',\
              'pussy', 'transgenders', 'transgender', 'fucking']
filler_words = ['engineering']

def remove_punctuation(text):
  no_punct = text.lower()
  no_punct = "".join([char for char in no_punct if char not in punct])
  no_punct = url_regex.sub('', no_punct)
  no_special = re.sub(r'[^a-zA-Z0-9\s]', '', no_punct)
  no_special = no_special.strip()


  return no_special

df['cleaned_comment'] = df['comment'].apply(remove_punctuation)

<ipython-input-12-dad6588b9cbc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.loc[(df.comment == '[removed]') | (df.comment == '[deleted]')]\


In [ ]:
#removing stop words
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
  tokens = word_tokenize(text)
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words
                     and word.isalpha() and word not in cuss_words and\
                     word not in filler_words]
  return " ".join(filtered_tokens)

df['cleaned_comment'] = df['cleaned_comment'].apply(remove_stopwords)
df['cleaned_comment'] = df['cleaned_comment'].str.replace('gon na', 'gonna')
df = df[df['cleaned_comment'] != '']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Lemmitization

In [ ]:
#lemmitization
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download the necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Create an instance of the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to lemmatize a single word based on its part of speech
def lemmatize_word(word, pos):
    """
    Lemmatize a word based on its part of speech.
    """
    # Map NLTK part of speech tags to WordNet part of speech tags
    if pos.startswith('J'):
        # Adjective
        pos_wordnet = wordnet.ADJ
    elif pos.startswith('V'):
        # Verb
        pos_wordnet = wordnet.VERB
    elif pos.startswith('N'):
        # Noun
        pos_wordnet = wordnet.NOUN
    elif pos.startswith('R'):
        # Adverb
        pos_wordnet = wordnet.ADV
    else:
        # Default to noun
        pos_wordnet = wordnet.NOUN

    # Lemmatize the word using the WordNetLemmatizer
    lemma = lemmatizer.lemmatize(word, pos=pos_wordnet)

    return lemma

# Define a function to lemmatize a sentence
def lemmatize_sentence(sentence):
    """
    Lemmatize a sentence.
    """
    # Tokenize the sentence into words
    words = nltk.word_tokenize(sentence)

    # Get the part of speech for each word
    pos_tags = nltk.pos_tag(words)

    # Lemmatize each word based on its part of speech
    lemmas = [lemmatize_word(word, pos) for word, pos in pos_tags]

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = ' '.join(lemmas)

    return lemmatized_sentence

# Apply the lemmatize_sentence function to the cleaned_comment column
df['cleaned_comment'] = df['cleaned_comment'].apply(lemmatize_sentence)
df.reset_index(inplace=True, drop=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Review cleaned text

In [ ]:
df['comment']

0       as one of the security guards there, the paren...
1       A bunch of elementary school kids (or maybe su...
2       "And here's the gym where your kid can pretend...
3       As a high school student who toured here, I ca...
4       Former Norlin guard here. Sounds like a case o...
                              ...                        
2466                            You’re fun at parties ...
2467    Anyone who played WoW at the time knew that im...
2468    That's not quite the expression and it definit...
2469    There’s a lot of people who don’t know that th...
2470    Even if you didnt play in Vanilla, if you were...
Name: comment, Length: 2471, dtype: object

In [ ]:
df['cleaned_comment'].head(20)

0     one security guard parent always glance quickl...
1     bunch elementary school kid maybe summer camp ...
2                             here gym kid pretend work
3           high school student tour confirm fact watch
4     former norlin guard sound like case watch east...
5                                                hahaha
6     say even first day new city way find college c...
7     stupid passage business worst investment cu ma...
8                              cough cough music center
9                             least isnt rubble anymore
10            graduate construction go literally decade
11                                              confirm
12                  hahaha wow applicable use meme ever
13                                         skooooo buff
14    shoutout whoever toss corn husk nebraska fan b...
15                                       wacky internet
16    ever since upgraded equipment obnoxiously slow...
17                                       buff bw

## Install packages

In [ ]:
%%capture
!pip install bertopic

## Use BerTopic model to do topic extraction!

I found it very handful and easy to use.
Here is the GIthub page. They also have a detailed documentation.
https://github.com/MaartenGr/BERTopic


In [ ]:
from bertopic import BERTopic
from umap import UMAP

# Make sure the results can be reproduced
umap_model = UMAP(n_neighbors=15, n_components=5,
                  min_dist=0.0, metric='cosine', random_state=43)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english")

topic_model = BERTopic(language="english",embedding_model = 'all-mpnet-base-v2',
                       vectorizer_model=vectorizer_model, n_gram_range=(1,3),
                       umap_model=umap_model,
                       calculate_probabilities=True, verbose=True)

topics, probs = topic_model.fit_transform(df['cleaned_comment'])

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

2023-05-03 21:29:26,083 - BERTopic - Transformed documents to Embeddings
2023-05-03 21:29:45,929 - BERTopic - Reduced dimensionality
2023-05-03 21:29:46,515 - BERTopic - Clustered reduced embeddings


In [ ]:
topic_model.reduce_topics(list(df['cleaned_comment']), nr_topics=100)

2023-05-03 21:29:47,150 - BERTopic - Reduced number of topics from 47 to 47


In [ ]:
#top post sorted by number in each topic
freq = topic_model.get_topic_info(); freq.head(30)

,Topic,Count,Name
0,-1,1114,-1_like_right_people_year
1,0,140,0_class_major_advisor_degree
2,1,111,1_football_team_game_point
3,2,61,2_food_chicken_eat_libby
4,3,60,3_boulder_colorado_city_scene
5,4,52,4_thanks_thank_kanye_nice
6,5,46,5_true_fact_agree_lol
7,6,45,6_wear_pillow_sock_jacket
8,7,42,7_lady_youre_guy_bos
9,8,39,8_snow_morning_day_blizzard


In [ ]:
#making a topic data frame
topic_df = topic_model.get_document_info(list(df['cleaned_comment']))
topic_df

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,one security guard parent always glance quickl...,10,10_cult_bible_group_church,cult - bible - group - church - book - study -...,0.007497,False
1,bunch elementary school kid maybe summer camp ...,-1,-1_like_right_people_year,like - right - people - year - im - think - do...,0.483267,False
2,here gym kid pretend work,0,0_class_major_advisor_degree,class - major - advisor - degree - business - ...,0.202878,False
3,high school student tour confirm fact watch,-1,-1_like_right_people_year,like - right - people - year - im - think - do...,0.724475,False
4,former norlin guard sound like case watch east...,30,30_library_east_music_cough,library - east - music - cough - feature - bui...,0.084606,False
...,...,...,...,...,...,...
2466,youre fun party,-1,-1_like_right_people_year,like - right - people - year - im - think - do...,0.023610,False
2467,anyone play wow time know immediately legendar...,-1,-1_like_right_people_year,like - right - people - year - im - think - do...,0.363169,False
2468,thats quite expression definitely doesnt apply...,15,15_joke_funny_laugh_hate,joke - funny - laugh - hate - reference - figu...,0.131079,False
2469,there lot people dont know though literally sc...,-1,-1_like_right_people_year,like - right - people - year - im - think - do...,0.098556,False


Look at the top words of one of the topic.

In [ ]:
topic_model.get_topic(8)

[('snow', 0.19323883516140192),
 ('morning', 0.08586968384359689),
 ('day', 0.05082317639769403),
 ('blizzard', 0.04400821125022368),
 ('power', 0.04195708058912051),
 ('idk', 0.04105959454354637),
 ('weather', 0.040813214927570875),
 ('start', 0.03995473001286391),
 ('cold', 0.03890325363397924),
 ('tuesday', 0.031178163769761517)]

In [ ]:
# Find topics related to word "class"
similar_topics, similarity = topic_model.find_topics("class", top_n=5); similar_topics

[0, 13, 27, 45, 44]

In [ ]:
# look at words in topic 87, which is related to word "covid"
topic_model.get_topic(8)

[('snow', 0.19323883516140192),
 ('morning', 0.08586968384359689),
 ('day', 0.05082317639769403),
 ('blizzard', 0.04400821125022368),
 ('power', 0.04195708058912051),
 ('idk', 0.04105959454354637),
 ('weather', 0.040813214927570875),
 ('start', 0.03995473001286391),
 ('cold', 0.03890325363397924),
 ('tuesday', 0.031178163769761517)]

## Visualizations

In [ ]:
topic_model.visualize_topics()

## Add Sentiment Analysis scores to df

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

sid = SentimentIntensityAnalyzer()

In [ ]:
#use inbuilt sid.polarity_scores to extract scores. Read on for the results
df['scores'] = df['cleaned_comment'].apply(lambda review: sid.polarity_scores(review))

#We break the dict generated above and pull only column 'compound'
df['compound']  = df['scores'].apply(lambda s : s['compound'])

#the step above returns values from -1 to 1.
df['comp_score'] = df['compound'].apply(lambda n : 'pos' if n >=0 else 'neg')


df.drop(df[df.compound == 0.0000].index, inplace=True)

df.head(5)

,ID,title,date,comment,comment_count,cleaned_comment,scores,compound,comp_score
0,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"as one of the security guards there, the paren...",179,one security guard parent always glance quickl...,"{'neg': 0.195, 'neu': 0.625, 'pos': 0.18, 'com...",-0.1027,neg
1,da859t,Bruh we’re being watched,2019-09-27 23:14:31,A bunch of elementary school kids (or maybe su...,160,bunch elementary school kid maybe summer camp ...,"{'neg': 0.077, 'neu': 0.811, 'pos': 0.113, 'co...",0.2023,pos
2,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"""And here's the gym where your kid can pretend...",60,here gym kid pretend work,"{'neg': 0.259, 'neu': 0.741, 'pos': 0.0, 'comp...",-0.1027,neg
4,da859t,Bruh we’re being watched,2019-09-27 23:14:31,Former Norlin guard here. Sounds like a case o...,80,former norlin guard sound like case watch east...,"{'neg': 0.167, 'neu': 0.652, 'pos': 0.181, 'co...",0.0516,pos
5,da859t,Bruh we’re being watched,2019-09-27 23:14:31,Hahaha,6,hahaha,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.5574,pos


In [ ]:
# pos vs negative comments
comp_dist = df.groupby('comp_score').size().reset_index(name = 'count').sort_values('count',ascending = False)

comp_dist.head()

,comp_score,count
1,pos,1052
0,neg,592


## Combine sentiment df and topic extraction information into one

In [ ]:
combined_df = pd.concat([df, topic_df], axis=1)

In [ ]:
combined_df.head()

,ID,title,date,comment,comment_count,cleaned_comment,scores,compound,comp_score,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"as one of the security guards there, the paren...",179.0,one security guard parent always glance quickl...,"{'neg': 0.195, 'neu': 0.625, 'pos': 0.18, 'com...",-0.1027,neg,one security guard parent always glance quickl...,10,10_cult_bible_group_church,cult - bible - group - church - book - study -...,0.007497,False
1,da859t,Bruh we’re being watched,2019-09-27 23:14:31,A bunch of elementary school kids (or maybe su...,160.0,bunch elementary school kid maybe summer camp ...,"{'neg': 0.077, 'neu': 0.811, 'pos': 0.113, 'co...",0.2023,pos,bunch elementary school kid maybe summer camp ...,-1,-1_like_right_people_year,like - right - people - year - im - think - do...,0.483267,False
2,da859t,Bruh we’re being watched,2019-09-27 23:14:31,"""And here's the gym where your kid can pretend...",60.0,here gym kid pretend work,"{'neg': 0.259, 'neu': 0.741, 'pos': 0.0, 'comp...",-0.1027,neg,here gym kid pretend work,0,0_class_major_advisor_degree,class - major - advisor - degree - business - ...,0.202878,False
4,da859t,Bruh we’re being watched,2019-09-27 23:14:31,Former Norlin guard here. Sounds like a case o...,80.0,former norlin guard sound like case watch east...,"{'neg': 0.167, 'neu': 0.652, 'pos': 0.181, 'co...",0.0516,pos,former norlin guard sound like case watch east...,30,30_library_east_music_cough,library - east - music - cough - feature - bui...,0.084606,False
5,da859t,Bruh we’re being watched,2019-09-27 23:14:31,Hahaha,6.0,hahaha,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.5574,pos,hahaha,5,5_true_fact_agree_lol,true - fact - agree - lol - yikes - uh - accur...,0.074665,False


In [ ]:
combined_df.to_csv('combined_df_2016.csv', index=False)